# Lesson 7: Making Sense of RNA-Seq Results

© 2020 David Gold. Except where the source is noted, this work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/).

## 7.1. Calculating Differential Expression

There are many different software packages avaialbe to detect differential expression in RNA-Seq studies. There is no consensus on which protocol is the best, and different programs will provide different sets of genes. This is partly because different packages use different methods to normalize the data and perform statistical tests.

Below is a table modified from [Seyednasrollah et al. (2015)](https://academic.oup.com/bib/article/16/1/59/240754) comparing the major packages. You can check out the paper to learn more about the strengths and weaknesses of each. As of 2019, edgeR, Cuffdiff2, and DESeq2 are the most popular programs ([McDermaid et al. 2019](https://academic.oup.com/bib/article/20/6/2044/5066173)).

|Method|Version|Normalization|Read count distribution assumption|Differential expression test|
|:---|:---|:---|:---|:---|
|edgeR|3.0.8|TMM/Upper quartile/RLE (DESeq-like)/None (all scaling factors are set to be one)|Negative binomial distribution|Exact test|
|DESeq|1.10.1|DESeq sizeFactors|Negative binomial distribution|Exact test|
|baySeq|1.12.0|Scaling factors (quantile/TMM/total)|Negative binomial distribution|Assesses the posterior probabilities of models for differentially and non-differentially expressed genes via empirical Bayesian methods and then compares these posterior likelihoods|
|NOIseq|1.1.4|RPKM/TMM/Upper quartile|Nonparametric method|Contrasts fold changes and absolute differences within a condition to determine the null distribution and then compares the observed differences to this null|
|SAMseq (samr)|2.0|SAMseq specialized method based on the mean read count over the null features of the data set|Nonparametric method|Wilcoxon rank statistic and a resampling strategy|
|Limma|3.14.4|TMM|voom transformation of counts|Empirical Bayes method|
|Cuffdiff 2 (Cufflinks)|2.0.2-beta|Geometric (DESeq-like)/quartile/classic-fpkm|Beta negative binomial distribution|t-test|
|EBSeq|1.1.7|DESeq median normalization|Negative binomial distribution|Evaluates the posterior probability of differentially and non-differentially expressed entities (genes or isoforms) via empirical Bayesian methods|

If you use [EdgeR](https://bioconductor.org/packages/release/bioc/html/edgeR.html) or [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html), the output  will look something like this:

|gene_id|sampleA|sampleB|logFC|logCPM|PValue|FDR|
|:---|:---|:---|:---|:---|:---|:---|
|XLOC_059513|Time1|Time2|5.0387959437543|5.90408721245344|1.49849535190046e-60|4.20567705464383e-56|
|XLOC_004889|Time1|Time2|-3.1335600954988|7.799294131704|1.1278428158703e-45|1.58270182351079e-41|
|XLOC_016950|Time1|Time2|-6.21826061399859|5.79427089198938|2.98853834019089e-45|2.79587723519326e-41|
|XLOC_037094|Time1|Time2|2.82459137179476|9.59372017044073|2.3639034038732e-40|1.65863282332763e-36|
|XLOC_054019|Time1|Time2|-6.77951756314346|5.55276623194202|8.13164669275025e-40|4.56445592157457e-36|
|XLOC_009950|Time1|Time2|3.90050134280392|5.16014818979294|1.63595455273615e-38|6.87823540384365e-35|
|XLOC_021797|Time1|Time2|-6.07665728256875|4.86195138316047|1.7155151367101e-38|6.87823540384365e-35|
|XLOC_021723|Time1|Time2|-3.57431209988049|9.71979967061391|1.25585975062605e-35|4.40586997013384e-32|
|XLOC_065496|Time1|Time2|-10.8710999929803|3.59379226474888|1.55662104184437e-34|4.85423624004489e-31|
|XLOC_050948|Time1|Time2|10.7667478027263|3.48720902928399|6.46418006289742e-34|1.81423677645279e-30|
|XLOC_036876|Time1|Time2|-3.30864407690335|5.96015993931641|1.1092930520264e-31|2.8303107998339e-28|
|XLOC_044612|Time1|Time2|-2.41875479358656|9.63116469527102|1.75699090699246e-31|4.10930889963752e-28|
|XLOC_004498|Time1|Time2|-5.21801688999268|4.02862477015151|4.53866835759368e-29|9.79863585570955e-26|
|XLOC_008915|Time1|Time2|-6.59155686056966|3.75065445768533|1.19840650472366e-28|2.40246264011245e-25|


In this analysis, differences in gene expression count as "signficiant" when they meet p-value and log fold change cutoffs. A false discovery rate adjusted p-vlaue (column __FDR__) of 0.05 or less is often used, or 0.001 if you want to be more stringent. A log fold change (column __logFC__) of 2 or more is an accepted cutoff.

If you use Cuffdiff from the [Cufflinks package](https://github.com/cole-trapnell-lab/cufflinks) the output will look something like this:

|test_id|gene_id|gene|locus|sample_1|sample_2|status|value_1|value_2|log2(fold_change)|test_stat|p_value|q_value|significant|
|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|
|XLOC_009988|XLOC_009988|LOC105332632|NC_047560.1:48399374-48401766|Time1|Time2|NOTEST|0.586789|0|-inf|0|1|1|no|
|XLOC_009989|XLOC_009989|LOC117687058|NC_047560.1:48439475-48440632|Time1|Time2|NOTEST|0|0|0|0|1|1|no|
|XLOC_009990|XLOC_009990|LOC117687026|NC_047560.1:48552059-48553441|Time1|Time2|OK|0|30.499|inf|-nan|5e-05|0.00080202|yes|
|XLOC_009992|XLOC_009992|LOC117687093|NC_047560.1:48570225-48574997|Time1|Time2|OK|0|164.604|inf|-nan|0.00195|0.0211162|yes|
|XLOC_009993|XLOC_009993|LOC105329686|NC_047560.1:48636544-48652405|Time1|Time2|NOTEST|0|0|0|0|1|1|no|
|XLOC_009994|XLOC_009994|LOC117680460|NC_047560.1:48636544-48652405|Time1|Time2|OK|40.3001|109.804|1.44608|1.36586|0.0293|0.174127|no|
|XLOC_009995|XLOC_009995|LOC117687564|NC_047560.1:48654750-48667405|Time1|Time2|NOTEST|0.528056|0.094631|-2.4803|0|1|no|
|XLOC_009996|XLOC_009996|LOC105329690,LOC117687567|NC_047560.1:48671214-48730660|Time1|Time2|OK|31.2578|48.4163|0.631278|1.129170.03865|0.20965|no|
|XLOC_009997|XLOC_009997|LOC105329691|NC_047560.1:48671214-48730660|Time1|Time2|OK|17.2188|0|-inf|-nan|0.08235|0.334674|no|
|XLOC_009998|XLOC_009998|LOC117687566|NC_047560.1:48671214-48730660|Time1|Time2|NOTEST|0|0|0|0|1|1|no|

Genes with a `yes` in the final column demonstrate a statistically significant difference between the two conditions (columns __value 1__ and __value 2__)

# 7.2. I've got my list of genes, now what?

Like many "big data" projects, there is a challange to finding a compelling story within the large volume of results. If you are comparing very different samples (say different life history stages within the same organism, or a multivariate study) you will probably have hundreds or thousands of differentially expressed genes.


# 7.3 Gene annotation

## 7.3.1. Find simmilar sequences using BLAST

The first step is to figure out what your differentially expressed genes are. If you are working on an organism with an annotated genome, then this has already been done for you. But if you are working on a new species (or if you re-did the gene models using a program like Cufflinks) then you will need to do some annotating yourself.

The simplest procedure is to use __BLAST__ (Forgot what BLAST is? See [Lesson 4](https://github.com/DavidGoldLab/Gold_Lab_Training/blob/master/Lesson_4_BLAST.ipynb)) to compare your genes to a database of curated sequences (for example, the [Uniprot/Swiss-Prot](https://www.uniprot.org/uniprot/?query=reviewed:yes) dataset). BLASTp will provide more accurate results than BLASTx, so we want to convert our RNA reads into protein predictions. This can be done with [TransDecoder](https://github.com/TransDecoder/TransDecoder/wiki):

In [ ]:
brew tap brewsci/bio
brew install transdecoder
TransDecoder.LongOrfs -t target_transcripts.fasta
TransDecoder.Predict -t target_transcripts.fasta

Next you can download the Uniprot/Swiss-Prot database, convert it into a BLAST database, and then peform BLASTP (protein vs protien) and BLASTX (DNA vs protein) analyses:

In [ ]:
# download the UNIPROT/SWISSPROT dataset
wget ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
# build BLAST database and perform BLAST
makeblastdb -in -dbtype prot -out UniprotDB
#blastx
blastx -query Trinity.fasta -db uniprot_sprot.pep -num_threads 8 -max_target_seqs 1 -outfmt 6 -evalue 1e-3 > blastx.outfmt6
#blastp
blastp -query transdecoder.pep -db uniprot_sprot.pep -num_threads 8 -max_target_seqs 1 -outfmt 6 -evalue 1e-3 > blastp.outfmt6

One thing to keep in mind is that this technique only tells you what gene in the Uniprot/Swiss-Prot database is most simmilar to your gene. It does not tell you that the two genes are homologous (remember, homology and simmilarity are not the same thing). To be confident that a gene is what you think it is, you'll want to do additional analyses.

## 7.3.2. Identify conserved protein domains with HMMER

BLAST will tell you if two genes are simmilar in their DNA/protein sequences, but it cannot tell you if two genes are likely to share a similar function. If two proteins from different organisms share conserved functional domains, that increases the chance that they play similar roles and are truly homologous. 

The [PFAM](www.pfam.org) database is the primary source for infromation on conserved protein domain sequences. We could download all the data in PFAM and use BLAST to find similar sequences in our queries. However, HMM searches are more common BUT WHY...

There is an [online server]() for small to medium number of proteins, but for a huge protein list you might want to run the program yourself:

In [ ]:
brew install hmmer
wget ftp://ftp.ebi.ac.uk/pub/databases/Pfam/current_release/Pfam-A.hmm.gz
gunzip Pfam-A.hmm.gz
hmmpress Pfam-A.hmm
hmmscan --cpu 12 --domtblout TrinotatePFAM.out Pfam-A.hmm transdecoder.pep > pfam.log

## 7.3.3. Additional annotation tools

### SignalP
This program predcts whether a protein sequence has one or more transmembrane domains (in other words, does the protein play a role in the cell membrane?). There is a [server]() you can use for up to 5,000 sequences. If you want to look at a larger dataset, it is better to run it locally. 

Unfortunatley, SignalP program cannot be downloaded through Homebrew. So you have to install it manually. Once you have it functioning you can run SignalP with the following commands:

In [ ]:
signalp -f short -n signalp.out transdecoder.pep

### RNAMMER
This program predcts whether a sequence is a ribosomal protein. Ribosomal proteins make up the vast majority of proteins in a cell, and represent contamination if we are interested in gene expression of messenger RNAs (mRNA). If a ribosomal protein is in your list of differentially expressed genes you should discard it. 

RNAMMER can be downloaded through Homebrew (in fact you should, because the program is a real pain in the butt to install manually):

In [ ]:
brew install brewsci/science/rnammer


# 7.4. Functional Annotation

## 7.4.1 GO Terms

The __Gene Ontology (GO)__ project is an initiative to unify terminology and gene attributes across species. It uses a standardized vocabulary and a hierarchical framework. There are three primary categories for GO terms:

- __Biological process (BP)__: The processes that a gene is known to play a role in (e.g. cell signaling, eye development, metabolism)
- __Molecular function (MF)__: The known activity of a gene product (e.g. cataylis, DNA binding)
- __Cellular component (CC)__: Where in a cell a gene is known to function (e.g. nucleous, ribosome)

## 7.4.2. KEGG Terms

The [Kyoto Encyclopedia of Gorilla? Genes](www.kegg.org) is a curated database of gene pathways. Most genes with an enzymatic or cell signaling function will have a KEGG ID.

## 7.4.3. EGGNOG Terms



# 7.5. Testing for enrichment

Enrichment testing is a broad swath of tests to see if your list of differentially expressed genes has a greater number of "X" than expected. That "X" could be a specific conserved protein domain, GO term, or KEGG pathway. Whether you have more of this term "than expected" is contingent on what you are comparing your list of genes to. This __background list__ of genes is usually all the genes in an organism (a.k.a. the transcriptome).  


[AmiGO 2](http://amigo.geneontology.org/amigo)

[DAVID: Database for Annotation, Visualization and Integrated Discovery](https://david.ncifcrf.gov/)

[Blast2GO](https://www.blast2go.com/) -- __Costs $$$ __


# 7.6. Putting it all together with Trinotate

[Trinotate](https://github.com/Trinotate/Trinotate.github.io/wiki) is a powerful set of scripts to take all of the information we collected in section 7.3. and organizes it into a searchable, SQL database. It also adds GO, KEGG, and EGNOGG info based on the results of the BLASTP search. This dataset can then be used to perform enrichment analyses using the R package GOSEQ (included with Trinotate and also available on its own).

# 7.7 (Notes)



In [ ]:
# Extract significant genes based on experiment
grep -e '.*\tL\tC2.*yes$' gene_exp.diff > 1_DEGenes_L_vs_C2
grep -e '.*\tL\tC3.*yes$' gene_exp.diff > 1_DEGenes_L_vs_C3
grep -e '.*\tC2\tC3.*yes$' gene_exp.diff > 1_DEGenes_C2_vs_C3
grep -e '.*\tL\tOA2.*yes$' gene_exp.diff > 1_DEGenes_L_vs_OA2
grep -e '.*\tC2\tOA2.*yes$' gene_exp.diff > 1_DEGenes_C2_vs_OA2
grep -e '.*\tC3\tOA2.*yes$' gene_exp.diff > 1_DEGenes_C3_vs_OA2
grep -e '.*\tL\tOA3.*yes$' gene_exp.diff > 1_DEGenes_L_vs_OA3
grep -e '.*\tC2\tOA3.*yes$' gene_exp.diff > 1_DEGenes_C2_vs_OA3
grep -e '.*\tC3\tOA3.*yes$' gene_exp.diff > 1_DEGenes_C3_vs_OA3
grep -e '.*\tOA2\tOA3.*yes$' gene_exp.diff > 1_DEGenes_OA2_vs_OA3
grep -e '.*\tL\tSM1.*yes$' gene_exp.diff > 1_DEGenes_L_vs_SM1
grep -e '.*\tC2\tSM1.*yes$' gene_exp.diff > 1_DEGenes_C2_vs_SM1
grep -e '.*\tC3\tSM1.*yes$' gene_exp.diff > 1_DEGenes_C3_vs_SM1
grep -e '.*\tOA2\tSM1.*yes$' gene_exp.diff > 1_DEGenes_OA2_vs_SM1
grep -e '.*\tOA3\tSM1.*yes$' gene_exp.diff > 1_DEGenes_OA3_vs_SM1
grep -e '.*\tL\tSM3.*yes$' gene_exp.diff > 1_DEGenes_L_vs_SM3
grep -e '.*\tC2\tSM3.*yes$' gene_exp.diff > 1_DEGenes_C2_vs_SM3
grep -e '.*\tC3\tSM3.*yes$' gene_exp.diff > 1_DEGenes_C3_vs_SM3
grep -e '.*\tOA2\tSM3.*yes$' gene_exp.diff > 1_DEGenes_OA2_vs_SM3
grep -e '.*\tOA3\tSM3.*yes$' gene_exp.diff > 1_DEGenes_OA3_vs_SM3
grep -e '.*\tSM1\tSM3.*yes$' gene_exp.diff > 1_DEGenes_SM1_vs_SM3

# Extract GO assignments from Trinotate Report

/Users/davidgold/Documents/bioinformatics/Trinotate-Trinotate-v3.2.0/util/extract_GO_assignments_from_Trinotate_xls.pl \
--Trinotate_xls Oyster_trinotate_annotation_report.9-25-20.xls \
-G --include_ancestral_terms \
> go_annotations.txt

# Create mapping file

awk -v OFS='\t' '{print $2,$1}' ../3_Cufflinks/cuffdiff/isoform_exp.diff > tmp
tail -n +2  tmp > Oyster_transcripts.fasta.gene_trans_map
rm tmp 

# Create transcripts lengths file

/usr/local/Cellar/trinity/2.8.3_1/libexec/util/misc/fasta_seq_length.pl  \
Oyster_transcripts.fasta > Oyster_transcripts.fasta.seq_lens

# Create gene lengths file

	# Must be run in Python2

python2 /usr/local/Cellar/trinity/2.8.3_1/libexec/util/misc/TPM_weighted_gene_length.py  \
--gene_trans_map Oyster_transcripts.fasta.gene_trans_map \
--trans_lengths Oyster_transcripts.fasta.seq_lens \
--TPM_matrix isoforms.fpkm_table > Oyster_transcripts.gene_lengths.txt

# Perform GO analysis in EdgeR

/usr/local/Cellar/trinity/2.8.3_1/libexec/Analysis/DifferentialExpression/run_DE_analysis.pl \
--matrix genes.count_table \
--method edgeR \
--samples_file sample_file.txt

cd edgeR.95289.dir

/usr/local/Cellar/trinity/2.8.3_1/libexec/Analysis/DifferentialExpression/analyze_diff_expr.pl \
--matrix ../genes.fpkm_table \
--samples ../sample_file.txt \
--examine_GO_enrichment --GO_annots ../GO_annotations.txt \
--gene_lengths ../Oyster_transcripts.gene_lengths.txt --include_GOplot

# Perform GOSeq on custom dataset

/usr/local/Cellar/trinity/2.8.3_1/libexec/Analysis/DifferentialExpression/run_GOseq.pl \
--factor_labeling  List_C2_vs_C3.txt \
--GO_assignments go_annotations.txt \
--lengths Oyster_transcripts.gene_lengths.txt \
--background background_all_genes.txt


__References:__

McDermaid, A., Monier, B., Zhao, J., Liu, B., & Ma, Q. (2019). Interpretation of differential gene expression results of RNA-seq data: review and integration. Briefings in bioinformatics, 20(6), 2044-2054.

Seyednasrollah, F., Laiho, A., & Elo, L. L. (2015). Comparison of software packages for detecting differential expression in RNA-seq studies. Briefings in bioinformatics, 16(1), 59-70.